<a href="https://colab.research.google.com/github/wallykop/IAD_SMMO_2019/blob/master/Kopylov_Chinese_characters_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ИАД KAGGLE картинки
Решение для [соревнования](https://www.kaggle.com/c/chinese-char-recognition-smmo19/overview) на курсе СММО ИАД 2019

[Папка](https://drive.google.com/drive/folders/1RvYnd7FASNHpFTkkiU5Z1Qv3LmyyV1So?usp=sharing) с файлами для загрузки 

In [0]:
!pip install tensorflow==2.0

     |████████████████████████████████| 86.3MB 43kB/s 
     |████████████████████████████████| 450kB 56.0MB/s 
     |████████████████████████████████| 3.8MB 61.3MB/s 
     |████████████████████████████████| 81kB 10.6MB/s 
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstall

In [0]:
import numpy as np 
import pandas as pd
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive')
tf.version

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


<module 'tensorflow_core._api.v2.version' from '/usr/local/lib/python3.6/dist-packages/tensorflow_core/_api/v2/version/__init__.py'>

In [0]:
data_train = np.load("/content/drive/My Drive/Chinese_characters_Kopylov/train-1.npy", allow_pickle=True)
for i in range(2, 5):
    t = np.load(f"/content/drive/My Drive/Chinese_characters_Kopylov/train-{i}.npy", allow_pickle=True)
    data_train = np.concatenate([data_train, t])
data_test = np.load("/content/drive/My Drive/Chinese_characters_Kopylov/test.npy", allow_pickle=True)

In [0]:
char_to_id = {char: id for id, char in enumerate (np.unique([char for _, char in data_train]))}
id_to_char = {id: char for id, char in enumerate (np.unique([char for _, char in data_train]))}

val_size = 0.2
batch_size = 128

def train_gen():
    for img, label in data_train[int(len(data_train) * val_size):]:
        img = img[..., None] 
        yield img, char_to_id[label]

def val_gen():
    for img, label in data_train[:int(len(data_train) * val_size)]:
        img = img[..., None] 
        yield img, char_to_id[label]

def test_gen():
    for img in data_test:
        img = img[:, :, np.newaxis] 
        yield img

def aug(img):
  tf.image.random_contrast(img)
  tf.image.random_brightness(img,max_delta=0.1)
  tf.image.random_saturation(img)

def preprocess_train(img, label):
  img = tf.image.resize(img, (64,64))
  img = tf.cast(img, tf.float32)
  img = (img - 127.5) / 127.5
  if np.random.rand() < 0.15:
    img = aug(img)
  label = tf.one_hot(label, 1000)
  return img, label

def preprocess_test(img):
  img = tf.image.resize(img, (64,64))
  img = tf.cast(img, tf.float32)
  img = (img - 127.5) / 127.5
  return img

dataset_train = tf.data.Dataset.from_generator(train_gen,
                                          output_types=(tf.float32, tf.int32),
                                          output_shapes=((None,None,1), ())
                                         ).map(preprocess_train, num_parallel_calls=-1).prefetch(-1).shuffle(1024).batch(batch_size).repeat()

dataset_val = tf.data.Dataset.from_generator(val_gen,
                                          output_types=(tf.float32, tf.int32),
                                          output_shapes=((None,None,1), ())
                                         ).map(preprocess_train, num_parallel_calls=-1).prefetch(-1).shuffle(1024).batch(batch_size).repeat()

dataset_test = tf.data.Dataset.from_generator(test_gen,
                                          output_types=tf.float32, 
                                          output_shapes=((None,None,1)),
                                         ).map(preprocess_test, num_parallel_calls=-1).prefetch(-1).batch(batch_size)

In [0]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, LeakyReLU, BatchNormalization
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.optimizers import adam
import cv2
from keras.models import save_model, load_model

class ModelSaveCallback(keras.callbacks.Callback):

    def __init__(self, file_name):
        super(ModelSaveCallback, self).__init__()
        self.file_name = file_name


    def on_epoch_end(self, epoch, logs=None):
        epoch += 1
        model.save(self.file_name.format(epoch))
        print(model.save(self.file_name.format(epoch)))

def load_from_file(model_filename, last_epoch):
  model = load_model(model_filename)
  epoch = last_epoch
  return model

Using TensorFlow backend.


In [0]:
def make_model():
    model = Sequential()
    
    model.add(Conv2D(filters=16, padding='same', kernel_size=(3,3), input_shape=(64,64,1), kernel_initializer='lecun_uniform', bias_initializer='lecun_uniform'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.1))

    model.add(Conv2D(filters=16, padding='same', kernel_size=(3,3), input_shape=(64,64,1), kernel_initializer='lecun_uniform', bias_initializer='lecun_uniform'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.1))

    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.5))



    model.add(Conv2D(filters=32, padding='same', kernel_size=(3,3), input_shape=(64,64,1), kernel_initializer='lecun_uniform', bias_initializer='lecun_uniform'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.1))

    model.add(Conv2D(filters=32, padding='same', kernel_size=(3,3), input_shape=(64,64,1), kernel_initializer='lecun_uniform', bias_initializer='lecun_uniform'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.1))

    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))



    model.add(Conv2D(filters=64, padding='same', kernel_size=(3,3), input_shape=(64,64,1), kernel_initializer='lecun_uniform', bias_initializer='lecun_uniform'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.1))

    model.add(Conv2D(filters=64, padding='same', kernel_size=(3,3), input_shape=(64,64,1), kernel_initializer='lecun_uniform', bias_initializer='lecun_uniform'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.1))

    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))



    model.add(Conv2D(filters=128, padding='same', kernel_size=(3,3), input_shape=(64,64,1), kernel_initializer='lecun_uniform', bias_initializer='lecun_uniform'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.1))

    model.add(Conv2D(filters=128, padding='same', kernel_size=(3,3), input_shape=(64,64,1), kernel_initializer='lecun_uniform', bias_initializer='lecun_uniform'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.1))

    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))



    model.add(Flatten())

    model.add(Dense(1024, kernel_initializer='lecun_uniform', bias_initializer='lecun_uniform'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.1))
    model.add(Dropout(0.1))

    model.add(Dense(1000, kernel_initializer='lecun_uniform', bias_initializer='lecun_uniform'))
    model.add(BatchNormalization())
    model.add(Activation("softmax"))
    return model

In [0]:
model = make_model()

print(model.summary())
model.compile(loss = "categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(lr = 0.001, decay = 0), metrics = ["accuracy"])

model_filename = 'weights_{0:02d}.hdf5'

def time_decay(epoch, initial_lrate):
    decay_rate = 0.01
    new_lrate = initial_lrate/(1+decay_rate*(epoch))
    return new_lrate

lrate = tf.keras.callbacks.LearningRateScheduler(time_decay,verbose=1)

def schedule(ind):
    a = [0.0001, 0.00005, 0.00005, 0.00001, 0.00001, 0.000005, 0.000001]
    return a[ind]

lr = tf.keras.callbacks.LearningRateScheduler(schedule)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                           patience=7,
                           verbose=1,
                           min_delta=1e-4)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor = 0.5, patience = 2, min_lr = 0.0001)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 64, 64, 16)        160       
_________________________________________________________________
batch_normalization_10 (Batc (None, 64, 64, 16)        64        
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 64, 64, 16)        2320      
_________________________________________________________________
batch_normalization_11 (Batc (None, 64, 64, 16)        64        
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 64, 64, 16)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 32, 16)       

In [0]:
history = model.fit(dataset_train, 
                    steps_per_epoch=500, 
                    epochs = 1,
                    validation_steps=100, 
                    validation_data=dataset_val, 
                    verbose = 1, 
                    callbacks = [ModelSaveCallback(model_filename), lr,
                                 #lrate, 
                                 reduce_lr,
                                 early_stop])

Train for 500 steps, validate for 100 steps
500/500 [==============================] - 1068s 2s/step - loss: 6.6449 - accuracy: 0.0092 - val_loss: 7.5027 - val_accuracy: 0.0011


In [0]:
submission = model.predict(dataset_test)
submission = submission.argmax(1)
result = pd.read_csv('/content/drive/My Drive/kaggle/random_labels.csv')
result['Category'] = submission
result['Category'] = result['Category'].apply(lambda x: id_to_char[x])
result.to_csv('predictions.csv', index = False)